In [1]:
"""Combine the automated and manual annotations"""

'Combine the automated and manual annotations'

In [2]:
import pandas as pd

In [3]:
basedir = '/scratch/ias41/ae_code/prev_reported_safety_associations'

In [4]:
all_formatted_effects = pd.read_excel(basedir + '/data/safety_original2formatted_effects.xls')

In [5]:
WBB_annotations = pd.read_csv(basedir + '/data/formatted_effects_WBB_accepted.txt', sep='\t')
manual_annotations_original = pd.read_csv(basedir + '/data/formatted_effects_manually_annotated.txt', sep='\t')
manual_annotations = manual_annotations_original.loc[~manual_annotations_original['MedDRA PT code'].isnull(),:].copy()
manual_annotations['MedDRA PT code'] = manual_annotations['MedDRA PT code'].astype(int)
manual_annotations.drop(labels=['Other level term', 'Parsing mistake', 'Comments', 'Other level code', 'Other level'], inplace=True, axis=1)
manual_annotations.columns = ['Formatted effect', 'PT Code', 'PT']

In [6]:
WBB_merge = all_formatted_effects.merge(WBB_annotations, left_on='Formatted effect', right_on='Row ID')
WBB_merge.drop(labels='Row ID', axis=1, inplace=True)

In [7]:
manual_merge = all_formatted_effects.merge(manual_annotations, on='Formatted effect')

In [8]:
total_merged = pd.concat([WBB_merge, manual_merge], sort=False)

In [9]:
total_merged.columns = ['Accession', 'ChEMBL target name', 'Original effect',
       'Formatted effect', 'PubMed ID', 'Reference', 'Annotated MedDRA PT',
       'Annotated MedDRA PT Code']
total_merged.sort_values(by=['ChEMBL target name', 'Formatted effect'], inplace=True)

In [10]:
total_merged.to_excel(basedir + '/data/safety_meddra_annotated_effects.xls', index=False)

### Save manually annotated HLTs

In [11]:
hlt_manual = manual_annotations_original.loc[manual_annotations_original['Other level']=='HLT',:]
hlt_manual.drop(labels=['Parsing mistake', 'Comments', 'MedDRA PT code', 'MedDRA PT','Other level'], inplace=True, axis=1)
hlt_manual_merged = all_formatted_effects.merge(hlt_manual, on='Formatted effect')
hlt_manual_merged['Other level code'] = hlt_manual_merged['Other level code'].astype(int)

/scratch/ias41/envs/release/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
hlt_manual_merged.columns = ['Accession', 'ChEMBL target name', 'Original effect',
       'Formatted effect', 'PubMed ID', 'Reference', 'Annotated MedDRA HLT Code',
       'Annotated MedDRA HLT']

In [13]:
hlt_manual_merged.to_excel(basedir + '/data/safety_meddra_manually_annotated_hlt_effects.xls', index=False)

### Save a copy of the PTs for hierarchy analysis in MedDRA browser

In [14]:
hierarchy_selection = total_merged[['Annotated MedDRA PT', 'Annotated MedDRA PT Code']].drop_duplicates().reset_index(drop=True)

In [15]:
hierarchy_selection['Row ID'] = ''
hierarchy_selection.columns = ['Term', 'Code', 'Row ID']

In [16]:
hierarchy_selection[['Row ID', 'Term', 'Code']].to_excel(basedir + '/data/safety_meddra_annotated_effects_for_hierarchy_input.xls', index=False)